In [3]:
import csv
import pandas as pd
import csv
import random
from collections import defaultdict


In [ ]:
# Superclass 1: precision & target sports
superclass_1 = {
    "air hockey": 1001,
    "archery": 1002,
    "axe throwing": 1003,
    "billiards": 1004,
    "bowling": 1005,
    "croquet": 1006,
    "curling": 1007,
    "disc golf": 1008,
    "golf": 1009,
    "horseshoe pitching": 1010,
    "shuffleboard": 1011
}
# Superclass 2: water
superclass_2 = {
    "canoe slamon": 2001,
    "fly fishing": 2002,
    "hydroplane racing": 2003,
    "log rolling": 2004,
    "rowing": 2005,
    "sailboat racing": 2006,
    "surfing": 2007,
    "swimming": 2008,
    "water cycling": 2009,
    "water polo": 2010
}

# Superclass 3: field & team ball sports
superclass_3 = {
    "ampute football": 3001,
    "baseball": 3002,
    "cricket": 3003,
    "field hockey": 3004,
    "football": 3005,
    "frisbee": 3006,
    "gaga": 3007,
    "lacrosse": 3008,
    "roller derby": 3009,
    "rugby": 3010,
    "ultimate": 3011
}

# Superclass 4: court ball sports
superclass_4 = {
    "basketball": 4001,
    "jai alai": 4002,
    "tennis": 4003,
    "volleyball": 4004,
    "wheelchair basketball": 4005,
    "table tennis": 4006
}

# Superclass 5: combat & strength sports
superclass_5 = {
    "arm wrestling": 5001,
    "boxing": 5002,
    "fencing": 5003,
    "jousting": 5004,
    "judo": 5005,
    "olympic wrestling": 5006,
    "rock climbing": 5007,
    "steer wrestling": 5008,
    "sumo wrestling": 5009,
    "tug of war": 5010,
    "weightlifting": 5011
}

# Superclass 6: equestrian & animal sports
superclass_6 = {
    "barell racing": 6001,
    "bull riding": 6002,
    "chuckwagon racing": 6003,
    "harness racing": 6004,
    "horse jumping": 6005,
    "horse racing": 6006,
    "polo": 6007
}

# Superclass 7: ice & snow sports
superclass_7 = {
    "bobsled": 7001,
    "figure skating men": 7002,
    "figure skating women": 7003,
    "figure skating pairs": 7004,
    "giant slalom": 7005,
    "hockey": 7006,
    "ice climbing": 7007,
    "ice yachting": 7008,
    "luge": 7009,
    "mushing": 7010,
    "ski jumping": 7011,
    "snow boarding": 7012,
    "snowmobile racing": 7013,
    "speed skating": 7014
}

# Superclass 8: motor & wheel racing
superclass_8 = {
    "bike polo": 8001,
    "bmx": 8002,
    "formula 1 racing": 8003,
    "motorcycle racing": 8004,
    "nascar racing": 8005,
    "rollerblade racing": 8006,
    "sidecar racing": 8007,
    "track bicycle": 8008,
    "wheelchair racing": 8009
}

# Superclass 9: gymnastics
superclass_9 = {
    "balance beam": 9001,
    "baton twirling": 9002,
    "cheerleading": 9003,
    "parallel bar": 9004,
    "pole dancing": 9005,
    "pommel horse": 9006,
    "rings": 9007,
    "trapeze": 9008,
    "uneven bars": 9009
}

# Superclass 10: aerial
superclass_10 = {
    "bungee jumping": 10001,
    "hang gliding": 10002,
    "sky surfing": 10003,
    "skydiving": 10004,
    "wingsuit flying": 10005
}

# Superclass 11: track & field athletics
superclass_11 = {
    "hammer throw": 11001,
    "high jump": 11002,
    "hurdles": 11003,
    "javelin": 11004,
    "pole climbing": 11005,
    "pole vault": 11006,
    "shot put": 11007
}

#number of classes of each superclass
num_classes = [0]*11
superclasses = [globals()[f"superclass_{i+1}"] for i in range(11)]
num_classes = [len(sc) for sc in superclasses]
print(num_classes)

[11, 10, 11, 6, 11, 7, 14, 9, 9, 5, 7]


In [ ]:
input_csv = "superclass_merged_sports.csv"

train_csv = "train.csv"
val_csv = "val.csv"
test_csv = "test.csv"

random.seed(42)   # ensures reproducibility

rows_by_class = defaultdict(list)

with open(input_csv, "r", newline="", encoding="utf-8") as infile:
    reader = csv.DictReader(infile)
    fieldnames = reader.fieldnames

    for row in reader:
        class_name = row["class"].strip()
        rows_by_class[class_name].append(row)

train_rows = []
val_rows = []
test_rows = []

for class_name, items in rows_by_class.items():
    random.shuffle(items)

    n = len(items)
    n_train = int(n * 0.70)
    n_val   = int(n * 0.15)
    n_test  = n - n_train - n_val  # ensures sum matches

    train_rows.extend(items[:n_train])
    val_rows.extend(items[n_train:n_train+n_val])
    test_rows.extend(items[n_train+n_val:])

def write_csv(filename, rows):
    with open(filename, "w", newline="", encoding="utf-8") as out:
        writer = csv.DictWriter(out, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)

write_csv(train_csv, train_rows)
write_csv(val_csv,  val_rows)
write_csv(test_csv, test_rows)

print("Done!")
print(f'Initial number of images {len(pd.read_csv(input_csv))}')
print(f"Train: {len(train_rows)}")
print(f"Validation: {len(val_rows)}")
print(f"Test: {len(test_rows)}")


Done!
initial numver of images 23993
Train: 16749
Validation: 3557
Test: 3687


In [ ]:
PERCENT_UNKNOWN = 0.40   # x% of class labels changed to "unknown"
train_csv = 'train.csv'
percent_int = int(PERCENT_UNKNOWN * 100)
unlabelled_train_csv = f'unlabelled_train_{percent_int}pct.csv'


def inject_unknown_labels(
        input_train_csv: str,
        output_csv: str,
        percent_unknown: float,
        seed: int = 42):
    """
    Convert a percentage of class labels in a training CSV to 'unknown'
    in a balanced way (same percent per class)

    Args:
        input_train_csv (str): path to the train CSV
        output_csv (str): output CSV with unknown labels injected
        percent_unknown (float): percentage (0-1) of samples per class to convert
        seed (int): random seed for reproducibility
    """

    random.seed(seed)

    # 1. Load and group by class
    rows_by_class = defaultdict(list)

    with open(input_train_csv, "r", newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        fieldnames = reader.fieldnames

        for row in reader:
            class_name = row["class"].strip()
            rows_by_class[class_name].append(row)

    # 2. Apply balanced unknown conversion
    final_rows = []

    for class_name, rows in rows_by_class.items():
        n = len(rows)
        n_unknown = int(n * percent_unknown)

        random.shuffle(rows)

        for i, row in enumerate(rows):
            if i < n_unknown:
                row["class"] = "unknown"
            final_rows.append(row)

    # 3. Save resulting CSV
    with open(output_csv, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(final_rows)

    print(f"[OK] Unknown labels injected.")
    print(f" → Input: {input_train_csv}")
    print(f" → Output: {output_csv}")
    print(f" → Unknown percentage per class: {percent_unknown * 100:.1f}%")
    print(f" → Total rows: {len(final_rows)}")

inject_unknown_labels(train_csv, unlabelled_train_csv, PERCENT_UNKNOWN)


[OK] Unknown labels injected.
 → Input: train.csv
 → Output: unlabelled_train_40pct.csv
 → Unknown percentage per class: 40.0%
 → Total rows: 16749
